<a href="https://colab.research.google.com/github/pps21april/PanScienceInnovationsProject/blob/main/Integrated_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install langchain_community
%pip install langchain_experimental
%pip install langchain-openai
%pip install langchain

In [ ]:
from google.colab import userdata


In [ ]:
%pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.5 MB/s eta 0:00:00


In [ ]:
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_experimental.text_splitter import SemanticChunker
import PyPDF2

In [ ]:
from google.colab import userdata

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.schema import Document
import sqlite3
import json

In [ ]:
def upload_documents(documents_folder_path):
    file_names = os.listdir(documents_folder_path)
    documents = []
    for filename in file_names:
        text = ""
        file_path = os.path.join(documents_folder_path, filename)
        if filename.endswith(".pdf"):
            with open(file_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                for page_num in range(len(pdf_reader.pages)):
                    page = pdf_reader.pages[page_num]
                    text += page.extract_text()
                    formatted_text = text.replace("\n", " ")
                    text_splitter = SemanticChunker(OpenAIEmbeddings(api_key=userdata.get("OPENAI_API_KEY")))
                    chunks = text_splitter.split_text(formatted_text)
                    for chunk in chunks:
                        doc = Document(page_content=chunk, metadata={"source": filename})
                        documents.append(doc)

    return documents

In [ ]:
def store_embeddings_create_retriever(documents_folder_path):
    embedding_function = OpenAIEmbeddings(model="text-embedding-ada-002",
                                      openai_api_key=userdata.get("OPENAI_API_KEY"))
    documents = upload_documents(documents_folder_path)
    vector_store = Chroma.from_documents(documents = documents,embedding = embedding_function)
    retriever = vector_store.as_retriever()
    return retriever

In [ ]:
def answering_user_queries(documents_folder_path,query):
    llm = ChatOpenAI(model_name="gpt-4o-mini", api_key=userdata.get("OPENAI_API_KEY"))
    retriever = store_embeddings_create_retriever(documents_folder_path)
    relevant_text = retriever.get_relevant_documents(query)

    prompt_template = """
    Answer user queries based on the context given in the documents below. Answers should be relevant,concise and relevant to the
    context . Return only the answer and nothing else. Always include sources of answer
    context : {context}
    query : {query}
    """

    prompt = PromptTemplate.from_template(prompt_template)
    rag_chain = prompt | llm
    response = rag_chain.invoke({"context":relevant_text,"query":query})
    return response.content

In [ ]:
query = "What is a transformer"
documents_folder_path = "documents_folder"

answering_user_queries(documents_folder_path,query)

<ipython-input-9-0c6b836f94a9>:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_text = retriever.get_relevant_documents(query)


'A Transformer is a network architecture based solely on attention mechanisms, designed for sequence transduction tasks, that eliminates the need for recurrence and convolutions. It is known for its superior quality in machine translation and is more parallelizable and requires less training time compared to traditional models. (Source: Transformers Research Paper.pdf)'

In [ ]:
def create_database_of_metadata(documents_folder_path,documents_folder_name):
    documents = upload_documents(documents_folder_path)

    # Connect to SQLite database (or create it)
    conn = sqlite3.connect("documents_metadata.db")
    cursor = conn.cursor()

    # Create table (if not exists)
    cursor.execute(f'''
    CREATE TABLE IF NOT EXISTS {documents_folder_name} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        content TEXT,
        metadata TEXT
    )
    ''')

    # Insert data
    for doc in documents:
        cursor.execute(f"INSERT INTO {documents_folder_name} (content, metadata) VALUES (?, ?)",
                       (doc.page_content, json.dumps(doc.metadata)))

    # Commit and close
    conn.commit()
    conn.close()



In [ ]:
create_database_of_metadata("documents_folder","documents_folder")

In [ ]:
conn = sqlite3.connect("documents_metadata.db")
cursor = conn.cursor()
cursor.execute("select * from documents_folder")
rows = cursor.fetchall()
conn.close()

rows[0:5]

[(1,
  'Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works. Attention Is All You Need Ashish Vaswani∗ Google Brain avaswani@google.comNoam Shazeer∗ Google Brain noam@google.comNiki Parmar∗ Google Research nikip@google.comJakob Uszkoreit∗ Google Research usz@google.com Llion Jones∗ Google Research llion@google.comAidan N. Gomez∗ † University of Toronto aidan@cs.toronto.eduŁukasz Kaiser∗ Google Brain lukaszkaiser@google.com Illia Polosukhin∗ ‡ illia.polosukhin@gmail.com Abstract The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. 